# T1059.003 - Windows Command Shell
Adversaries may abuse the Windows command shell for execution. The Windows command shell ([cmd](https://attack.mitre.org/software/S0106)) is the primary command prompt on Windows systems. The Windows command prompt can be used to control almost any aspect of a system, with various permission levels required for different subsets of commands. The command prompt can be invoked remotely via [Remote Services](https://attack.mitre.org/techniques/T1021) such as [SSH](https://attack.mitre.org/techniques/T1021/004).(Citation: SSH in Windows)

Batch files (ex: .bat or .cmd) also provide the shell with a list of sequential commands to run, as well as normal scripting operations such as conditionals and loops. Common uses of batch files include long or repetitive tasks, or the need to run the same set of commands on multiple systems.

Adversaries may leverage [cmd](https://attack.mitre.org/software/S0106) to execute various commands and payloads. Common uses include [cmd](https://attack.mitre.org/software/S0106) to execute a single command, or abusing [cmd](https://attack.mitre.org/software/S0106) interactively with input and output forwarded over a command and control channel.

## Atomic Tests

### Atomic Test #1 - Create and Execute Batch Script
Creates and executes a simple batch script. Upon execution, CMD will briefly launch to run the batch script then close again.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Batch file must exist on disk at specified location (#{script_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\T1059.003_script.bat") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item "PathToAtomicsFolder\..\ExternalPayloads\T1059.003_script.bat" -Force | Out-Null
Set-Content -Path "PathToAtomicsFolder\..\ExternalPayloads\T1059.003_script.bat" -Value "dir"

```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process "PathToAtomicsFolder\..\ExternalPayloads\T1059.003_script.bat"
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\T1059.003_script.bat" -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Writes text to a file and displays it.Writes text to a file and display the results. This test is intended to emulate the dropping of a malicious file to disk.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
echo "Hello from the Windows Command Prompt!" > "%TEMP%\test.bin" & type "%TEMP%\test.bin"
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 2

#### Cleanup: 
```cmd
del "%TEMP%\test.bin" >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Suspicious Execution via Windows Command ShellCommand line executed via suspicious invocation. Example is from the 2021 Threat Detection Report by Red Canary.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
%LOCALAPPDATA:~-3,1%md /c echo Hello, from CMD! > hello.txt & type hello.txt
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 3

### Atomic Test #4 - Simulate BlackByte Ransomware Print Bombing
This test attempts to open a file a specified number of times in Wordpad, then prints the contents. 
It is designed to mimic BlackByte ransomware's print bombing technique, where tree.dll, which contains the ransom note, is opened in Wordpad 75 times and then printed. 
See https://redcanary.com/blog/blackbyte-ransomware/. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: File to print must exist on disk at specified location (#{file_to_print})

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\T1059_003note.txt"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
new-item "PathToAtomicsFolder\..\ExternalPayloads\T1059_003note.txt" -value "This file has been created by T1059.003 Test 4" -Force | Out-Null

```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cmd /c "for /l %x in (1,1,75) do start wordpad.exe /p PathToAtomicsFolder\..\ExternalPayloads\T1059_003note.txt" | out-null
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 4

#### Cleanup: 
```powershell
stop-process -name wordpad -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Command Prompt read contents from CMD file and execute
Simulate Raspberry Robin using the "standard-in" command prompt feature cmd `/R <` to read and execute a file via cmd.exe
See https://redcanary.com/blog/raspberry-robin/. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: CMD file must exist on disk at specified location (#{input_file})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1059.003\src\t1059.003_cmd.cmd") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1059.003\src\t1059.003_cmd.cmd") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1059.003/src/t1059.003_cmd.cmd" -OutFile "PathToAtomicsFolder\T1059.003\src\t1059.003_cmd.cmd"

```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
cmd /r cmd<"PathToAtomicsFolder\T1059.003\src\t1059.003_cmd.cmd"
```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 5

### Atomic Test #6 - Command prompt writing script to file then executes it    Simulate DarkGate malware's second stage by writing a VBscript to disk directly from the command prompt then executing it.
    The script will execute 'whoami' then exit.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
 c:\windows\system32\cmd.exe /c cd /d %TEMP%\ & echo Set objShell = CreateObject("WScript.Shell"):Set objExec = objShell.Exec("whoami"):Set objExec = Nothing:Set objShell = Nothing > AtomicTest.vbs & AtomicTest.vbs```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 6

#### Cleanup: 
```cmd
del "AtomicTest.vbs" >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1059.003 -TestNumbers 6 -Cleanup

## Detection
Usage of the Windows command shell may be common on administrator, developer, or power user systems depending on job function. If scripting is restricted for normal users, then any attempt to enable scripts running on a system would be considered suspicious. If scripts are not commonly used on a system, but enabled, scripts running out of cycle from patching or other administrator functions are suspicious. Scripts should be captured from the file system when possible to determine their actions and intent.

Scripts are likely to perform actions with various effects on a system that may generate events, depending on the types of monitoring used. Monitor processes and command-line arguments for script execution and subsequent behavior. Actions may be related to network and system information Discovery, Collection, or other scriptable post-compromise behaviors and could be used as indicators of detection leading back to the source script.